In [5]:
!pip install gurobipy
import gurobipy as gp
from gurobipy import GRB

def solve_distribution():
    # 1. Definir os dados do problema
    factories = ['Liverpool', 'Brighton']
    depots = ['Newcastle', 'Birmingham', 'London', 'Exeter']
    customers = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6']

    capacity = {'Liverpool': 150000, 'Brighton': 200000}
    throughput = {'Newcastle': 70000, 'Birmingham': 50000,
                  'London': 100000, 'Exeter': 40000}
    demand = {'C1': 50000, 'C2': 10000, 'C3': 40000,
              'C4': 35000, 'C5': 60000, 'C6': 20000}

    # Custos de transporte
    costs = {
        ('Liverpool', 'Newcastle'): 0.5, ('Liverpool', 'Birmingham'): 0.5,
        ('Liverpool', 'London'): 1.0, ('Liverpool', 'Exeter'): 0.2,
        ('Brighton', 'Birmingham'): 0.3, ('Brighton', 'London'): 0.5,
        ('Brighton', 'Exeter'): 0.2,
        ('Liverpool', 'C1'): 1.0, ('Liverpool', 'C3'): 1.5,
        ('Liverpool', 'C4'): 2.0, ('Liverpool', 'C6'): 1.0,
        ('Brighton', 'C1'): 2.0, ('Brighton', 'C2'): 1.5,
        ('Newcastle', 'C2'): 1.5, ('Newcastle', 'C3'): 0.5,
        ('Newcastle', 'C4'): 1.5, ('Newcastle', 'C6'): 1.0,
        ('Birmingham', 'C1'): 1.0, ('Birmingham', 'C2'): 0.5,
        ('Birmingham', 'C3'): 0.5, ('Birmingham', 'C5'): 0.5,
        ('London', 'C3'): 2.0, ('London', 'C4'): 1.0,
        ('London', 'C5'): 0.5, ('London', 'C6'): 1.5,
        ('Exeter', 'C3'): 0.2, ('Exeter', 'C4'): 1.5,
        ('Exeter', 'C5'): 0.5, ('Exeter', 'C6'): 1.5
    }

    # Definir os arcos (rotas) possiveis
    factory_depot_arcs, cost_fd = gp.multidict({
        ('Liverpool', 'Newcastle'): 0.5, ('Liverpool', 'Birmingham'): 0.5,
        ('Liverpool', 'London'): 1.0, ('Liverpool', 'Exeter'): 0.2,
        ('Brighton', 'Birmingham'): 0.3, ('Brighton', 'London'): 0.5,
        ('Brighton', 'Exeter'): 0.2 })

    factory_customer_arcs, cost_fc = gp.multidict({
        ('Liverpool', 'C1'): 1.0, ('Liverpool', 'C3'): 1.5,
        ('Liverpool', 'C4'): 2.0, ('Liverpool', 'C6'): 1.0,
        ('Brighton', 'C1'): 2.0, ('Brighton', 'C2'): 1.5 })

    depot_customer_arcs, cost_dc = gp.multidict({
        ('Newcastle', 'C2'): 1.5, ('Newcastle', 'C3'): 0.5, ('Newcastle', 'C4'): 1.5,
        ('Newcastle', 'C6'): 1.0, ('Birmingham', 'C1'): 1.0, ('Birmingham', 'C2'): 0.5,
        ('Birmingham', 'C3'): 0.5, ('Birmingham', 'C5'): 0.5, ('London', 'C3'): 2.0,
        ('London', 'C4'): 1.0, ('London', 'C5'): 0.5, ('London', 'C6'): 1.5,
        ('Exeter', 'C3'): 0.2, ('Exeter', 'C4'): 1.5, ('Exeter', 'C5'): 0.5,
        ('Exeter', 'C6'): 1.5 })

    # --- Modelo 1: Custo Mínimo ---
    m1 = gp.Model("Distribution_Base")

    # 2. Criar as variáveis de decisão
    x = m1.addVars(factory_depot_arcs, name="Fab_Dep")
    y = m1.addVars(factory_customer_arcs, name="Fab_Cli")
    z = m1.addVars(depot_customer_arcs, name="Dep_Cli")

    # 3. Definir a função objetivo
    m1.setObjective(x.prod(cost_fd) + y.prod(cost_fc) + z.prod(cost_dc), GRB.MINIMIZE)

    # 4. Adicionar as restrições
    m1.addConstrs((x.sum(f, '*') + y.sum(f, '*') <= capacity[f] for f in factories), "Capacity")
    m1.addConstrs((y.sum('*', c) + z.sum('*', c) == demand[c] for c in customers), "Demand")
    m1.addConstrs((x.sum('*', d) == z.sum(d, '*') for d in depots), "Depot_Flow")
    m1.addConstrs((x.sum('*', d) <= throughput[d] for d in depots), "Throughput")

    # 5. Otimizar o modelo
    m1.optimize()

    # --- Modelo 2: Com Preferências ---
    m2 = m1.copy()
    m2.set_name("Distribution_Preferences")

    # Recuperar ponteiros para as variáveis no modelo copiado
    y2 = gp.tupledict({arc: m2.getVarByName(y[arc].VarName) for arc in factory_customer_arcs})
    z2 = gp.tupledict({arc: m2.getVarByName(z[arc].VarName) for arc in depot_customer_arcs})

    # Adicionar restrições de preferência
    m2.addConstr(y2['Brighton', 'C1'] + z2['Birmingham', 'C1'] == 0, "Pref_C1")
    m2.addConstr(y2['Brighton', 'C2'] + z2['Birmingham', 'C2'] == 0, "Pref_C2")
    m2.addConstr(z2['London', 'C5'] + z2['Exeter', 'C5'] == 0, "Pref_C5")
    m2.addConstr(y2['Liverpool', 'C6'] + z2['Newcastle', 'C6'] == 0, "Pref_C6")

    m2.optimize()

    # 6. Exibir os resultados (pode ser adaptado para retornar os valores)
    print("--- Modelo Base (Custo Minimo) ---")
    if m1.Status == GRB.OPTIMAL:
        print(f"Custo Total: {m1.ObjVal:,.2f}")

    print("\n--- Modelo com Preferencias ---")
    if m2.Status == GRB.OPTIMAL:
        print(f"Custo com Preferencias: {m2.ObjVal:,.2f}")
        extra_cost = m2.ObjVal - m1.ObjVal
        print(f"Custo Adicional: {extra_cost:,.2f}")

# Chamar a funcao principal para executar a otimizacao
solve_distribution()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 16 rows, 29 columns and 65 nonzeros
Model fingerprint: 0x32f4f9fa
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 15 rows, 29 columns, 64 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4800000e+05   1.812500e+04   0.000000e+00      0s
      10    1.9850000e+05   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.985000000e+05


AttributeError: 'gurobipy.Model' object has no attribute 'set_name'